# Imports

In [1]:
import os
import gc
import pickle

In [2]:
from flair.data import Sentence
from flair.models import SequenceTagger
from flair.models import TextClassifier

from flair.data import TaggedCorpus
from flair.data_fetcher import  NLPTaskDataFetcher, NLPTask

import torch

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


In [4]:
from pymongo import MongoClient

client = MongoClient()

db = client['glvis_db']

# Extract hidden representations from flair's pretrained NER model

In [6]:
ner_tagger = SequenceTagger.load('ner')

2019-03-23 17:18:52,068 loading file /home/snie/.flair/models/en-ner-conll03-v0.4.pt


In [7]:
ner_tagger

SequenceTagger(
  (embeddings): StackedEmbeddings(
    (list_embedding_0): WordEmbeddings()
    (list_embedding_1): FlairEmbeddings(
      (lm): LanguageModel(
        (drop): Dropout(p=0.05)
        (encoder): Embedding(300, 100)
        (rnn): LSTM(100, 2048)
        (decoder): Linear(in_features=2048, out_features=300, bias=True)
      )
    )
    (list_embedding_2): FlairEmbeddings(
      (lm): LanguageModel(
        (drop): Dropout(p=0.05)
        (encoder): Embedding(300, 100)
        (rnn): LSTM(100, 2048)
        (decoder): Linear(in_features=2048, out_features=300, bias=True)
      )
    )
  )
  (word_dropout): WordDropout()
  (locked_dropout): LockedDropout()
  (embedding2nn): Linear(in_features=4196, out_features=4196, bias=True)
  (rnn): LSTM(4196, 256, bidirectional=True)
  (linear): Linear(in_features=512, out_features=20, bias=True)
)

In [8]:
corpus = NLPTaskDataFetcher.load_corpus(NLPTask.CONLL_03, base_path='data/conll/')

2019-03-23 17:19:06,783 Reading data from data/conll/conll_03
2019-03-23 17:19:06,784 Train: data/conll/conll_03/eng.train
2019-03-23 17:19:06,784 Dev: data/conll/conll_03/eng.testa
2019-03-23 17:19:06,784 Test: data/conll/conll_03/eng.testb


In [9]:
tag_dictionary = corpus.make_tag_dictionary(tag_type='ner')

In [10]:
len(corpus.get_all_sentences())

22137

## First linear layer

In [5]:
db_col = db['flair_ner_embedding2nn']

In [ ]:
db_col.drop()

In [ ]:
for i, sentence in enumerate(corpus.get_all_sentences()):
    print(f'Start sentence {i}')
    
    # Define hook to get intermediate values
    hidden_states = torch.zeros(len(sentence), 1, 4196)
    def hook(m, i):
        hidden_states.copy_(i[0].data)
                                
    h = ner_tagger.embedding2nn.register_forward_pre_hook(hook)
                                
    ner_tagger.predict(sentence)
                                
    spans = sentence.get_spans('ner')
                                
    # Informaction to store: the named entities, their predicted labels, probabilities and hidden states
    # If there are multiple words for one entity, take the average value of hidden states
    # and record the number of words in the entity
    
    for span in spans:
        entry = {}
        entry['text'] = span.text
        entry['tag'] = span.tag
        entry['score'] = span.score
        entry['token_num'] = len(span.tokens)
        
        idx = [token.idx-1 for token in span.tokens]
        entry['embedding2nn'] = hidden_states[idx, :, :].mean(dim=0).squeeze().tolist()
        
        db_col.insert_one(entry)
    
    h.remove()
    
    print(f'Finish sentence {i}')

Start sentence 0
Finish sentence 0
Start sentence 1
Finish sentence 1
Start sentence 2
Finish sentence 2
Start sentence 3
Finish sentence 3
Start sentence 4
Finish sentence 4
Start sentence 5
Finish sentence 5
Start sentence 6
Finish sentence 6
Start sentence 7
Finish sentence 7
Start sentence 8
Finish sentence 8
Start sentence 9
Finish sentence 9
Start sentence 10
Finish sentence 10
Start sentence 11
Finish sentence 11
Start sentence 12
Finish sentence 12
Start sentence 13
Finish sentence 13
Start sentence 14
Finish sentence 14
Start sentence 15
Finish sentence 15
Start sentence 16
Finish sentence 16
Start sentence 17
Finish sentence 17
Start sentence 18
Finish sentence 18
Start sentence 19
Finish sentence 19
Start sentence 20
Finish sentence 20
Start sentence 21
Finish sentence 21
Start sentence 22
Finish sentence 22
Start sentence 23
Finish sentence 23
Start sentence 24
Finish sentence 24
Start sentence 25
Finish sentence 25
Start sentence 26
Finish sentence 26
Start sentence 27
Fin

Finish sentence 216
Start sentence 217
Finish sentence 217
Start sentence 218
Finish sentence 218
Start sentence 219
Finish sentence 219
Start sentence 220
Finish sentence 220
Start sentence 221
Finish sentence 221
Start sentence 222
Finish sentence 222
Start sentence 223
Finish sentence 223
Start sentence 224
Finish sentence 224
Start sentence 225
Finish sentence 225
Start sentence 226
Finish sentence 226
Start sentence 227
Finish sentence 227
Start sentence 228
Finish sentence 228
Start sentence 229
Finish sentence 229
Start sentence 230
Finish sentence 230
Start sentence 231
Finish sentence 231
Start sentence 232
Finish sentence 232
Start sentence 233
Finish sentence 233
Start sentence 234
Finish sentence 234
Start sentence 235
Finish sentence 235
Start sentence 236
Finish sentence 236
Start sentence 237
Finish sentence 237
Start sentence 238
Finish sentence 238
Start sentence 239
Finish sentence 239
Start sentence 240
Finish sentence 240
Start sentence 241
Finish sentence 241
Start

Finish sentence 427
Start sentence 428
Finish sentence 428
Start sentence 429
Finish sentence 429
Start sentence 430
Finish sentence 430
Start sentence 431
Finish sentence 431
Start sentence 432
Finish sentence 432
Start sentence 433
Finish sentence 433
Start sentence 434
Finish sentence 434
Start sentence 435
Finish sentence 435
Start sentence 436
Finish sentence 436
Start sentence 437
Finish sentence 437
Start sentence 438
Finish sentence 438
Start sentence 439
Finish sentence 439
Start sentence 440
Finish sentence 440
Start sentence 441
Finish sentence 441
Start sentence 442
Finish sentence 442
Start sentence 443
Finish sentence 443
Start sentence 444
Finish sentence 444
Start sentence 445
Finish sentence 445
Start sentence 446
Finish sentence 446
Start sentence 447
Finish sentence 447
Start sentence 448
Finish sentence 448
Start sentence 449
Finish sentence 449
Start sentence 450
Finish sentence 450
Start sentence 451
Finish sentence 451
Start sentence 452
Finish sentence 452
Start

Finish sentence 639
Start sentence 640
Finish sentence 640
Start sentence 641
Finish sentence 641
Start sentence 642
Finish sentence 642
Start sentence 643
Finish sentence 643
Start sentence 644
Finish sentence 644
Start sentence 645
Finish sentence 645
Start sentence 646
Finish sentence 646
Start sentence 647
Finish sentence 647
Start sentence 648
Finish sentence 648
Start sentence 649
Finish sentence 649
Start sentence 650
Finish sentence 650
Start sentence 651
Finish sentence 651
Start sentence 652
Finish sentence 652
Start sentence 653
Finish sentence 653
Start sentence 654
Finish sentence 654
Start sentence 655
Finish sentence 655
Start sentence 656
Finish sentence 656
Start sentence 657
Finish sentence 657
Start sentence 658
Finish sentence 658
Start sentence 659
Finish sentence 659
Start sentence 660
Finish sentence 660
Start sentence 661
Finish sentence 661
Start sentence 662
Finish sentence 662
Start sentence 663
Finish sentence 663
Start sentence 664
Finish sentence 664
Start

Finish sentence 851
Start sentence 852
Finish sentence 852
Start sentence 853
Finish sentence 853
Start sentence 854
Finish sentence 854
Start sentence 855
Finish sentence 855
Start sentence 856
Finish sentence 856
Start sentence 857
Finish sentence 857
Start sentence 858
Finish sentence 858
Start sentence 859
Finish sentence 859
Start sentence 860
Finish sentence 860
Start sentence 861
Finish sentence 861
Start sentence 862
Finish sentence 862
Start sentence 863
Finish sentence 863
Start sentence 864
Finish sentence 864
Start sentence 865
Finish sentence 865
Start sentence 866
Finish sentence 866
Start sentence 867
Finish sentence 867
Start sentence 868
Finish sentence 868
Start sentence 869
Finish sentence 869
Start sentence 870
Finish sentence 870
Start sentence 871
Finish sentence 871
Start sentence 872
Finish sentence 872
Start sentence 873
Finish sentence 873
Start sentence 874
Finish sentence 874
Start sentence 875
Finish sentence 875
Start sentence 876
Finish sentence 876
Start

...

Finish sentence 8938
Start sentence 8939
Finish sentence 8939
Start sentence 8940
Finish sentence 8940
Start sentence 8941
Finish sentence 8941
Start sentence 8942
Finish sentence 8942
Start sentence 8943
Finish sentence 8943
Start sentence 8944
Finish sentence 8944
Start sentence 8945
Finish sentence 8945
Start sentence 8946
Finish sentence 8946
Start sentence 8947
Finish sentence 8947
Start sentence 8948
Finish sentence 8948
Start sentence 8949
Finish sentence 8949
Start sentence 8950
Finish sentence 8950
Start sentence 8951
Finish sentence 8951
Start sentence 8952
Finish sentence 8952
Start sentence 8953
Finish sentence 8953
Start sentence 8954
Finish sentence 8954
Start sentence 8955
Finish sentence 8955
Start sentence 8956
Finish sentence 8956
Start sentence 8957
Finish sentence 8957
Start sentence 8958
Finish sentence 8958
Start sentence 8959
Finish sentence 8959
Start sentence 8960
Finish sentence 8960
Start sentence 8961
Finish sentence 8961
Start sentence 8962
Finish sentence 

Finish sentence 9138
Start sentence 9139
Finish sentence 9139
Start sentence 9140
Finish sentence 9140
Start sentence 9141
Finish sentence 9141
Start sentence 9142
Finish sentence 9142
Start sentence 9143
Finish sentence 9143
Start sentence 9144
Finish sentence 9144
Start sentence 9145
Finish sentence 9145
Start sentence 9146
Finish sentence 9146
Start sentence 9147
Finish sentence 9147
Start sentence 9148
Finish sentence 9148
Start sentence 9149
Finish sentence 9149
Start sentence 9150
Finish sentence 9150
Start sentence 9151
Finish sentence 9151
Start sentence 9152
Finish sentence 9152
Start sentence 9153
Finish sentence 9153
Start sentence 9154
Finish sentence 9154
Start sentence 9155
Finish sentence 9155
Start sentence 9156
Finish sentence 9156
Start sentence 9157
Finish sentence 9157
Start sentence 9158
Finish sentence 9158
Start sentence 9159
Finish sentence 9159
Start sentence 9160
Finish sentence 9160
Start sentence 9161
Finish sentence 9161
Start sentence 9162
Finish sentence 

Finish sentence 9341
Start sentence 9342
Finish sentence 9342
Start sentence 9343
Finish sentence 9343
Start sentence 9344
Finish sentence 9344
Start sentence 9345
Finish sentence 9345
Start sentence 9346
Finish sentence 9346
Start sentence 9347
Finish sentence 9347
Start sentence 9348
Finish sentence 9348
Start sentence 9349
Finish sentence 9349
Start sentence 9350
Finish sentence 9350
Start sentence 9351
Finish sentence 9351
Start sentence 9352
Finish sentence 9352
Start sentence 9353
Finish sentence 9353
Start sentence 9354
Finish sentence 9354
Start sentence 9355
Finish sentence 9355
Start sentence 9356
Finish sentence 9356
Start sentence 9357
Finish sentence 9357
Start sentence 9358
Finish sentence 9358
Start sentence 9359
Finish sentence 9359
Start sentence 9360
Finish sentence 9360
Start sentence 9361
Finish sentence 9361
Start sentence 9362
Finish sentence 9362
Start sentence 9363
Finish sentence 9363
Start sentence 9364
Finish sentence 9364
Start sentence 9365
Finish sentence 

Finish sentence 9541
Start sentence 9542
Finish sentence 9542
Start sentence 9543
Finish sentence 9543
Start sentence 9544
Finish sentence 9544
Start sentence 9545
Finish sentence 9545
Start sentence 9546
Finish sentence 9546
Start sentence 9547
Finish sentence 9547
Start sentence 9548
Finish sentence 9548
Start sentence 9549
Finish sentence 9549
Start sentence 9550
Finish sentence 9550
Start sentence 9551
Finish sentence 9551
Start sentence 9552
Finish sentence 9552
Start sentence 9553
Finish sentence 9553
Start sentence 9554
Finish sentence 9554
Start sentence 9555
Finish sentence 9555
Start sentence 9556
Finish sentence 9556
Start sentence 9557
Finish sentence 9557
Start sentence 9558
Finish sentence 9558
Start sentence 9559
Finish sentence 9559
Start sentence 9560
Finish sentence 9560
Start sentence 9561
Finish sentence 9561
Start sentence 9562
Finish sentence 9562
Start sentence 9563
Finish sentence 9563
Start sentence 9564
Finish sentence 9564
Start sentence 9565
Finish sentence 

Finish sentence 9745
Start sentence 9746
Finish sentence 9746
Start sentence 9747
Finish sentence 9747
Start sentence 9748
Finish sentence 9748
Start sentence 9749
Finish sentence 9749
Start sentence 9750
Finish sentence 9750
Start sentence 9751
Finish sentence 9751
Start sentence 9752
Finish sentence 9752
Start sentence 9753
Finish sentence 9753
Start sentence 9754
Finish sentence 9754
Start sentence 9755
Finish sentence 9755
Start sentence 9756
Finish sentence 9756
Start sentence 9757
Finish sentence 9757
Start sentence 9758
Finish sentence 9758
Start sentence 9759
Finish sentence 9759
Start sentence 9760
Finish sentence 9760
Start sentence 9761
Finish sentence 9761
Start sentence 9762
Finish sentence 9762
Start sentence 9763
Finish sentence 9763
Start sentence 9764
Finish sentence 9764
Start sentence 9765
Finish sentence 9765
Start sentence 9766
Finish sentence 9766
Start sentence 9767
Finish sentence 9767
Start sentence 9768
Finish sentence 9768
Start sentence 9769
Finish sentence 

## Last linear layer

In [ ]:
db_col = db['flair_ner_linear']

In [ ]:
for i, sentence in enumerate(corpus.get_all_sentences()):
    print(f'Start sentence {i}')
    
    # Define hook to get intermediate values
    hidden_states = torch.zeros(len(sentence), 1, 512)
    def hook(m, i):
        hidden_states.copy_(i[0].data)
                                
    h = ner_tagger.linear.register_forward_pre_hook(hook)
                                
    ner_tagger.predict(sentence)
                                
    spans = sentence.get_spans('ner')
                                
    # Informaction to store: the named entities, their predicted labels, probabilities and hidden states
    # If there are multiple words for one entity, take the average value of hidden states
    # and record the number of words in the entity
    
    for span in spans:
        entry = {}
        entry['text'] = span.text
        entry['tag'] = span.tag
        entry['score'] = span.score
        entry['token_num'] = len(span.tokens)
        
        idx = [token.idx-1 for token in span.tokens]
        entry['linear_layer_state'] = hidden_states[idx, :, :].mean(dim=0).squeeze().tolist()
        
        db_col.insert_one(entry)
    
    h.remove()
    
    print(f'Finish sentence {i}')